In [1]:
from VecSim import *
import numpy as np
import hnswlib

In [2]:
dim = 16
num_elements = 10000
space = 'l2'
M=16
efConstruction = 100
efRuntime = 10

### Build RedisLabs VecSim HNSWLib Index

In [3]:
params = VecSimParams()
hnswparams = HNSWParams()

params.algo = VecSimAlgo_HNSWLIB

hnswparams.dim = dim
hnswparams.metric = VecSimMetric_L2
hnswparams.type = VecSimType_FLOAT32
hnswparams.M = M
hnswparams.efConstruction = efConstruction
hnswparams.initialCapacity = num_elements
hnswparams.efRuntime=efRuntime

params.hnswParams = hnswparams
index = VecSimIndex(params)

### Build HNSWLib index

In [4]:
p = hnswlib.Index(space='l2', dim=dim)
p.init_index(max_elements=num_elements, ef_construction=efConstruction, M=M)
p.set_ef(efRuntime)

### Load data

In [5]:
data = np.float32(np.random.random((num_elements, dim)))
for i, vector in enumerate(data):
    index.add_vector(vector, i)
    p.add_items(vector, i)

### Test

In [6]:
import unittest
from  numpy.testing import assert_array_equal

class TestNotebook(unittest.TestCase):
    
    def test_index(self):
        for vector in data:
            hnswlib_labels, hnswlib_distances = p.knn_query(vector, k=10)
            redis_labels, redis_distances = index.knn_query(vector, 10)
            assert_array_equal(hnswlib_labels, redis_labels)
            assert_array_equal(hnswlib_distances, redis_distances)
        

unittest.main(argv=[''], verbosity=2, exit=False)

test_index (__main__.TestNotebook) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.790s

OK
